This notebook is used to run liana for CrossTalkeR according to the documentation provided by the authors:
[Run liana for Ligand Receptor Interaction Prediction](https://costalab.github.io/CrossTalkeR/articles/run_liana.html)

In [ ]:
import liana as li
import scanpy as sc
import os
import logging
import tomlkit
import pandas as pd
from os import path
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)
logging.basicConfig(level=logging.ERROR)
# sc.set_figure_params(frameon=False, dpi=300)
sc._settings.ScanpyConfig.n_jobs = -1

In [ ]:
## Pipeline parameters
with open("../config.toml", "r") as f:
    config = tomlkit.parse(f.read())

ROOT_DIR = config["basic"]["ANALYSIS_DIR"]
DIR_SAVE = path.join(ROOT_DIR, config["basic"]["DIR_SAVE"])
TISSUE = config["basic"]["TISSUE"]
ANNOTATION_METHOD = config["annotation"]["ANNOTATION_METHOD"]
COUNTS_LAYER = config["normalization"]["COUNTS_LAYER"]
NORMAMALIZATION_LAYER = config["normalization"]["NORMALIZATION_METHOD"]

In [ ]:
adata = sc.read_h5ad(path.join(DIR_SAVE, "adata.h5ad"))
adata.raw = adata

In [ ]:
sc.pl.umap(adata, color='annotation', title='', frameon=False)

In [ ]:
# li.resource.show_resources()
resource = li.rs.select_resource('consensus')
resource.head()

If the reference genome is mouse, please use the following code to generate mouse resource.

In [ ]:
if False:
    map_df = li.rs.get_hcop_orthologs(url='https://ftp.ebi.ac.uk/pub/databases/genenames/hcop/human_mouse_hcop_fifteen_column.txt.gz',
                                    columns=['human_symbol', 'mouse_symbol'],
                                    # NOTE: HCOP integrates multiple resource, so we can filter out mappings in at least 3 of them for confidence
                                    min_evidence=3
                                    )
    # rename the columns to source and target, respectively for the original organism and the target organism
    map_df = map_df.rename(columns={'human_symbol':'source', 'mouse_symbol':'target'})

    # We will then translate
    mouse = li.rs.translate_resource(resource,
                                    map_df=map_df,
                                    columns=['ligand', 'receptor'],
                                    replace=True,
                                    # Here, we will be harsher and only keep mappings that don't map to more than 1 mouse gene
                                    one_to_many=1
                                    )
    mouse

In [ ]:
if not os.path.exists("./res_liana/"):
    os.makedirs("./res_liana/")
for i in set(adata.obs['sample']):
    # print(i)
    lr = li.method.cellphonedb(adata[adata.obs['sample']==i],
                          groupby='annotation',
                          expr_prop=0.1,
                          #resource=mouse,
                          verbose=True,
                          resource_name='consensus',
                          inplace=False)
    lr.to_csv(f"./res_liana/{i}_lr_liana_consensus_unfiltered.csv")

In [ ]:
data = {}
for i in os.listdir("./res_liana/"):
    if i.endswith('lr_liana_consensus_unfiltered.csv'):
        evfull = pd.read_csv(i)
        evfull = evfull.loc[:,['ligand','receptor','source','target','lr_means','cellphone_pvals']]
        evfull['type_gene_A'] = 'Ligand'
        evfull['type_gene_B'] = 'Receptor'
        evfull['gene_A'] = evfull['ligand']
        evfull['gene_B'] = evfull['receptor']
        evfull['MeanLR'] = evfull['lr_means']
        evfull = evfull.loc[list(evfull.cellphone_pvals.to_numpy()<=0.05),:]
        k=i[0:i.find('_lr_')]
        evfull.loc[:, ['source', 'target', 'type_gene_A', 'type_gene_B', 'gene_A', 'gene_B', 'MeanLR']].to_csv(f'./res_liana/{k}_lr_ready.csv', index=False)
        data[k]=os.path.abspath(f'./res_liana/{k}_lr_ready.csv')